# Exercise 5b

In [1]:
import numpy as np

In [2]:
# Direction: row number to column number 
transition = [[0, 1/2, 0, 1/2, 0, 0],
              [0, 0, 1, 0, 0, 0],
              [1/2, 0, 0, 0, 1/2, 0],
              [0, 0, 0, 0, 0, 1],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0]    
]
transition = np.array(transition)

In [3]:
teleportation = [0.1, 0.2, 0.4]
for alpha in teleportation:
    state = np.array([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
    for i in range(100):
        state = alpha/len(state) + (1-alpha)*np.dot(state, transition)
#         print('iteration-{}, state: {}'.format(i, state))
#         print()
    print('alpha-{}, state:{}'.format(alpha, state))

alpha-0.1, state:[0.03780699 0.03367981 0.0469785  0.29965352 0.29552634 0.28635484]
alpha-0.2, state:[0.06574924 0.05963303 0.08103976 0.27370031 0.2675841  0.25229358]
alpha-0.4, state:[0.1042988  0.09795631 0.12544045 0.23537703 0.22903453 0.20789288]


In [4]:
import nltk
nltk.download('book')

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\pablo\AppData\R

True

In [5]:
from nltk.corpus import wordnet as wn 
import random

In [6]:
# Find the synsets of the word horror
wn.synsets('horror')

[Synset('horror.n.01'), Synset('horror.n.02'), Synset('repugnance.n.01')]

In [7]:
# Find the first synset
horror = wn.synset('horror.n.01')

In [8]:
# Find the hypernyms of the horror object
prueba = horror.hypernyms()
prueba = prueba[0]
prueba = prueba.lemma_names()
print(prueba[0])

fear


In [9]:
from nltk.corpus import movie_reviews 
import random

In [10]:
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [16]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = set(list(all_words)[:2000]) # We only focus on the most frequent 2000 words.
# Use one-hot encoding as features. 1 means the given word is contained in the document and 0 otherwise.
def document_features(document): # [_document-classify-extractor]
    document_words = set(document) # [_document-classify-set]
    features = {}
    for word in word_features:
        word_synset = wn.synsets(word)
        if len(word_synset)>0:
            word_synset = word_synset[0]
            word_hyp = word_synset.hypernyms()
            if len(word_hyp)>0:
                word_hyp = word_hyp[0]
                word_hyp = word_hyp.lemma_names()
                word_hyp = word_hyp[0]
                features['contains({})'.format(word_hyp)] = word_hyp in document_words
            else:
                features['contains({})'.format(word)] = word in document_words
        else:
            features['contains({})'.format(word)] = word in document_words
            
    return features

In [17]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 
classifier.show_most_informative_features(5)

0.8
Most Informative Features
     contains(atrocious) = True              neg : pos    =     11.7 : 1.0
 contains(unimaginative) = True              neg : pos    =      8.4 : 1.0
      contains(reminder) = True              pos : neg    =      8.3 : 1.0
    contains(schumacher) = True              neg : pos    =      6.6 : 1.0
        contains(suvari) = True              neg : pos    =      6.4 : 1.0


### Change the feature into occurrences of the words but not just if the document contains certain words

In [8]:
def document_features_occurrences(document): # [_document-classify-extractor]
    features = {}
    for word in word_features:
        features[word] = 0
    for word_ in document:
        if word_ in word_features:
            features[word_] += 1
    return features

In [13]:
featuresets = [(document_features_occurrences(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 
classifier.show_most_informative_features(5)

0.79
Most Informative Features
              apparently = 2                 neg : pos    =     13.8 : 1.0
                  boring = 2                 neg : pos    =     11.9 : 1.0
                     bad = 5                 neg : pos    =     11.1 : 1.0
                     ugh = 1                 neg : pos    =      8.4 : 1.0
                      we = 8                 pos : neg    =      8.2 : 1.0


### Use other classifier 

In [15]:
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC

In [16]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classif = SklearnClassifier(SVC(), sparse=False).train(train_set)
print(nltk.classify.accuracy(classif, test_set)) 

0.87


In [17]:
featuresets = [(document_features_occurrences(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classif = SklearnClassifier(SVC(), sparse=False).train(train_set)
print(nltk.classify.accuracy(classif, test_set)) 

0.79


It seems that use the of occurences of the words which are contained in the documents is not a better choice in this case. We can also use tf-idf encoding as feature. Now this is up to you for exploration. 